In [4]:
pip install requests

  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached idna-3.11-py3-none-any.whl.metadata (8.4 kB)
Using cached requests-2.32.5-py3-none-any.whl (64 kB)
Using cached idna-3.11-py3-none-any.whl (71 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [requests]4/5 [requests]
Note: you may need to restart the kernel to use updated packages.


In [8]:
from __future__ import annotations

import json
import os
from datetime import date, timedelta
from pathlib import Path
from typing import Any, Dict, Optional

# Shared config
BASE_URL = "https://api.football-data.org/v4"
TOKEN_ENV = "FOOTBALL_DATA_API_TOKEN"  # football-data.org API key
MAX_LIST_ITEMS = 10


def compact(obj: Any, *, max_items: int = MAX_LIST_ITEMS) -> Any:
    """Compact lists/dicts so prints stay readable (first N items only)."""
    if isinstance(obj, dict):
        out: Dict[str, Any] = {}
        for k, v in obj.items():
            if isinstance(v, list):
                out[k] = v[:max_items]
                if len(v) > max_items:
                    out[k].append(f"... ({len(v) - max_items} more)")
            else:
                out[k] = compact(v, max_items=max_items)
        return out
    if isinstance(obj, list):
        out = obj[:max_items]
        if len(obj) > max_items:
            out = out + [f"... ({len(obj) - max_items} more)"]
        return out
    return obj


def print_result(title: str, payload: Any) -> None:
    print(f"\n=== {title} ===")
    print(json.dumps(compact(payload), indent=2, ensure_ascii=False))


try:
    import requests  # type: ignore
except Exception as e:
    raise ImportError("Install requests: pip install requests") from e


def _load_dotenv_simple() -> None:
    """Load key/value pairs from a local .env file into os.environ (no deps).

    Searches for .env in:
    - current working directory
    - repo root (one level up from this notebook's folder)
    """

    candidates = [Path.cwd() / ".env", Path.cwd().parent / ".env"]
    # Also consider repo root relative to this notebook (AIA/code/parser.ipynb -> AIA/.env)
    try:
        here = Path(__file__).resolve()  # may not exist in notebooks
        candidates.append(here.parent.parent / ".env")
    except Exception:
        pass

    env_path = next((p for p in candidates if p.exists()), None)
    if not env_path:
        return

    for raw_line in env_path.read_text(encoding="utf-8").splitlines():
        line = raw_line.strip()
        if not line or line.startswith("#"):
            continue
        if line.startswith("export "):
            line = line[len("export ") :].lstrip()
        if "=" not in line:
            continue
        k, v = line.split("=", 1)
        k = k.strip()
        v = v.strip().strip('"').strip("'")
        if k and k not in os.environ:
            os.environ[k] = v


_load_dotenv_simple()

token = os.getenv(TOKEN_ENV)
if not token:
    raise EnvironmentError(
        f"Missing {TOKEN_ENV}. Your notebook does not see it. "
        f"Set it in .env or export it in your shell."
    )

session = requests.Session()
session.headers.update({"X-Auth-Token": token})


def api_get(path: str, *, params: Optional[Dict[str, Any]] = None) -> Any:
    url = f"{BASE_URL}{path}"
    r = session.get(url, params=params, timeout=30)
    r.raise_for_status()
    return r.json()



In [9]:
# Areas: list + single area
areas_resp = api_get("/areas")
print_result("GET /areas (first 10)", areas_resp)

areas = areas_resp.get("areas") or []
area_id = (areas[0] or {}).get("id") if areas else None

if area_id is not None:
    area_resp = api_get(f"/areas/{area_id}")
    print_result(f"GET /areas/{area_id}", area_resp)
else:
    print("\n=== GET /areas/{id} ===\nSkipped (no areas returned)")




=== GET /areas (first 10) ===
{
  "count": 272,
  "filters": {},
  "areas": [
    {
      "id": 2000,
      "name": "Afghanistan",
      "countryCode": "AFG",
      "flag": null,
      "parentAreaId": 2014,
      "parentArea": "Asia"
    },
    {
      "id": 2001,
      "name": "Africa",
      "countryCode": "AFR",
      "flag": null,
      "parentAreaId": 2267,
      "parentArea": "World"
    },
    {
      "id": 2002,
      "name": "Albania",
      "countryCode": "ALB",
      "flag": null,
      "parentAreaId": 2077,
      "parentArea": "Europe"
    },
    {
      "id": 2004,
      "name": "Algeria",
      "countryCode": "ALG",
      "flag": null,
      "parentAreaId": 2001,
      "parentArea": "Africa"
    },
    {
      "id": 2005,
      "name": "American Samoa",
      "countryCode": "ASM",
      "flag": null,
      "parentAreaId": 2175,
      "parentArea": "Oceania"
    },
    {
      "id": 2006,
      "name": "Andorra",
      "countryCode": "AND",
      "flag": null,
      "pare

In [10]:
# Competitions: list + single competition (hardcoded example: PL)
competitions_resp = api_get("/competitions")
print_result("GET /competitions (first 10)", competitions_resp)

pl_competition = api_get("/competitions/PL")
print_result("GET /competitions/PL", pl_competition)




=== GET /competitions (first 10) ===
{
  "count": 13,
  "filters": {
    "client": "Eamtc"
  },
  "competitions": [
    {
      "id": 2013,
      "area": {
        "id": 2032,
        "name": "Brazil",
        "code": "BRA",
        "flag": "https://crests.football-data.org/764.svg"
      },
      "name": "Campeonato Brasileiro Série A",
      "code": "BSA",
      "type": "LEAGUE",
      "emblem": "https://crests.football-data.org/bsa.png",
      "plan": "TIER_ONE",
      "currentSeason": {
        "id": 2474,
        "startDate": "2026-01-28",
        "endDate": "2026-12-02",
        "currentMatchday": 1,
        "winner": null
      },
      "numberOfAvailableSeasons": 10,
      "lastUpdated": "2024-09-13T16:55:53Z"
    },
    {
      "id": 2016,
      "area": {
        "id": 2072,
        "name": "England",
        "code": "ENG",
        "flag": "https://crests.football-data.org/770.svg"
      },
      "name": "Championship",
      "code": "ELC",
      "type": "LEAGUE",
      "embl

In [11]:
# Competition / Standings
pl_standings = api_get("/competitions/PL/standings")
print_result("GET /competitions/PL/standings", pl_standings)




=== GET /competitions/PL/standings ===
{
  "filters": {
    "season": "2025"
  },
  "area": {
    "id": 2072,
    "name": "England",
    "code": "ENG",
    "flag": "https://crests.football-data.org/770.svg"
  },
  "competition": {
    "id": 2021,
    "name": "Premier League",
    "code": "PL",
    "type": "LEAGUE",
    "emblem": "https://crests.football-data.org/PL.png"
  },
  "season": {
    "id": 2403,
    "startDate": "2025-08-15",
    "endDate": "2026-05-24",
    "currentMatchday": 21,
    "winner": null
  },
  "standings": [
    {
      "stage": "REGULAR_SEASON",
      "type": "TOTAL",
      "group": null,
      "table": [
        {
          "position": 1,
          "team": {
            "id": 57,
            "name": "Arsenal FC",
            "shortName": "Arsenal",
            "tla": "ARS",
            "crest": "https://crests.football-data.org/57.png"
          },
          "playedGames": 21,
          "form": null,
          "won": 15,
          "draw": 4,
          "lost": 2

In [12]:
# Competition / Matches (API doesn't expose limit here; we still print first 10 via print_result)
recent_to = date.today()
recent_from = recent_to - timedelta(days=14)

pl_matches = api_get(
    "/competitions/PL/matches",
    params={"dateFrom": recent_from.isoformat(), "dateTo": recent_to.isoformat()},
)
print_result(
    f"GET /competitions/PL/matches?dateFrom={recent_from.isoformat()}&dateTo={recent_to.isoformat()} (first 10)",
    pl_matches,
)




=== GET /competitions/PL/matches?dateFrom=2025-12-27&dateTo=2026-01-10 (first 10) ===
{
  "filters": {
    "season": "2025"
  },
  "resultSet": {
    "count": 39,
    "first": "2025-12-27",
    "last": "2026-01-08",
    "played": 39
  },
  "competition": {
    "id": 2021,
    "name": "Premier League",
    "code": "PL",
    "type": "LEAGUE",
    "emblem": "https://crests.football-data.org/PL.png"
  },
  "matches": [
    {
      "area": {
        "id": 2072,
        "name": "England",
        "code": "ENG",
        "flag": "https://crests.football-data.org/770.svg"
      },
      "competition": {
        "id": 2021,
        "name": "Premier League",
        "code": "PL",
        "type": "LEAGUE",
        "emblem": "https://crests.football-data.org/PL.png"
      },
      "season": {
        "id": 2403,
        "startDate": "2025-08-15",
        "endDate": "2026-05-24",
        "currentMatchday": 21,
        "winner": null
      },
      "id": 537963,
      "utcDate": "2025-12-27T12:30:00

In [13]:
# Competition / Teams
pl_teams = api_get("/competitions/PL/teams")
print_result("GET /competitions/PL/teams (first 10)", pl_teams)




=== GET /competitions/PL/teams (first 10) ===
{
  "count": 20,
  "filters": {
    "season": "2025"
  },
  "competition": {
    "id": 2021,
    "name": "Premier League",
    "code": "PL",
    "type": "LEAGUE",
    "emblem": "https://crests.football-data.org/PL.png"
  },
  "season": {
    "id": 2403,
    "startDate": "2025-08-15",
    "endDate": "2026-05-24",
    "currentMatchday": 21,
    "winner": null
  },
  "teams": [
    {
      "area": {
        "id": 2072,
        "name": "England",
        "code": "ENG",
        "flag": "https://crests.football-data.org/770.svg"
      },
      "id": 57,
      "name": "Arsenal FC",
      "shortName": "Arsenal",
      "tla": "ARS",
      "crest": "https://crests.football-data.org/57.png",
      "address": "75 Drayton Park London N5 1BU",
      "website": "http://www.arsenal.com",
      "founded": 1886,
      "clubColors": "Red / White",
      "venue": "Emirates Stadium",
      "runningCompetitions": [
        {
          "id": 2021,
          "nam

In [14]:
# Competition / (Top)Scorers (supports limit)
pl_scorers = api_get("/competitions/PL/scorers", params={"limit": 10})
print_result("GET /competitions/PL/scorers?limit=10", pl_scorers)




=== GET /competitions/PL/scorers?limit=10 ===
{
  "count": 10,
  "filters": {
    "season": "2025",
    "limit": 10
  },
  "competition": {
    "id": 2021,
    "name": "Premier League",
    "code": "PL",
    "type": "LEAGUE",
    "emblem": "https://crests.football-data.org/PL.png"
  },
  "season": {
    "id": 2403,
    "startDate": "2025-08-15",
    "endDate": "2026-05-24",
    "currentMatchday": 21,
    "winner": null
  },
  "scorers": [
    {
      "player": {
        "id": 38101,
        "name": "Erling Haaland",
        "firstName": "Erling",
        "lastName": "Haaland",
        "dateOfBirth": "2000-07-21",
        "nationality": "Norway",
        "section": "Centre-Forward",
        "position": null,
        "shirtNumber": null,
        "lastUpdated": "2025-04-02T09:41:03Z"
      },
      "team": {
        "id": 65,
        "name": "Manchester City FC",
        "shortName": "Man City",
        "tla": "MCI",
        "crest": "https://crests.football-data.org/65.png",
        "ad

In [15]:
# Teams: list (first page) + team details + team matches (supports limit)
teams_resp = api_get("/teams", params={"limit": 10, "offset": 0})
print_result("GET /teams?limit=10&offset=0", teams_resp)

teams = teams_resp.get("teams") or []
team_id = (teams[0] or {}).get("id") if teams else None

if team_id is not None:
    team_resp = api_get(f"/teams/{team_id}")
    print_result(f"GET /teams/{team_id}", team_resp)

    team_matches = api_get(f"/teams/{team_id}/matches", params={"limit": 10})
    print_result(f"GET /teams/{team_id}/matches?limit=10", team_matches)
else:
    print("\n=== Team details ===\nSkipped (no teams returned)")




=== GET /teams?limit=10&offset=0 ===
{
  "count": 10,
  "filters": {
    "limit": 10,
    "offset": 0,
    "permission": "TIER_ONE"
  },
  "teams": [
    {
      "id": 1,
      "name": "1. FC Köln",
      "shortName": "1. FC Köln",
      "tla": "KOE",
      "crest": "https://crests.football-data.org/1.png",
      "address": "Franz-Kremer-Allee 1 Köln 50937",
      "website": "http://www.fc-koeln.de",
      "founded": 1948,
      "clubColors": "Red / White",
      "venue": "RheinEnergieSTADION",
      "lastUpdated": "2022-02-25T16:49:46Z"
    },
    {
      "id": 2,
      "name": "TSG 1899 Hoffenheim",
      "shortName": "Hoffenheim",
      "tla": "TSG",
      "crest": "https://crests.football-data.org/2.png",
      "address": "Horrenberger Straße 58 Zuzenhausen 74939",
      "website": "http://www.achtzehn99.de",
      "founded": 1921,
      "clubColors": "Blue / White",
      "venue": "PreZero Arena",
      "lastUpdated": "2022-02-25T16:49:58Z"
    },
    {
      "id": 3,
      "name

In [18]:
# Matches: cross-competition list (hardcode Premier League id = 2021), then match details + head2head
# Note: /matches doesn't support limit; we print first 10 via print_result.
pl_competition_id = 2021

matches_resp = api_get(
    "/matches",
    params={
        "competitions": pl_competition_id,
        # "dateFrom": recent_from.isoformat(),
        # "dateTo": recent_to.isoformat(),
    },
)
print_result(
    f"GET /matches?competitions={pl_competition_id}&dateFrom={recent_from.isoformat()}&dateTo={recent_to.isoformat()} (first 10)",
    matches_resp,
)

matches = matches_resp.get("matches") or []
match_id = (matches[0] or {}).get("id") if matches else None

if match_id is not None:
    match_detail = api_get(f"/matches/{match_id}")
    print_result(f"GET /matches/{match_id}", match_detail)

    h2h = api_get(f"/matches/{match_id}/head2head", params={"limit": 10})
    print_result(f"GET /matches/{match_id}/head2head?limit=10", h2h)
else:
    print("\n=== Match details ===\nSkipped (no matches returned for date window)")




=== GET /matches?competitions=2021&dateFrom=2025-12-27&dateTo=2026-01-10 (first 10) ===
{
  "filters": {
    "dateFrom": "2026-01-10",
    "dateTo": "2026-01-11",
    "permission": "TIER_ONE",
    "competitions": "2021"
  },
  "resultSet": {
    "count": 0
  },
  "matches": []
}

=== Match details ===
Skipped (no matches returned for date window)


In [19]:
# Persons: best-effort example using a referee id from the match detail (if present)
if "match_detail" in globals() and isinstance(match_detail, dict):
    referees = (match_detail.get("match") or {}).get("referees") or []
    person_id = (referees[0] or {}).get("id") if referees else None
else:
    person_id = None

if person_id is not None:
    person_resp = api_get(f"/persons/{person_id}")
    print_result(f"GET /persons/{person_id}", person_resp)

    person_matches = api_get(
        f"/persons/{person_id}/matches",
        params={"limit": 10, "dateFrom": recent_from.isoformat(), "dateTo": recent_to.isoformat()},
    )
    print_result(
        f"GET /persons/{person_id}/matches?limit=10&dateFrom={recent_from.isoformat()}&dateTo={recent_to.isoformat()}",
        person_matches,
    )
else:
    print("\n=== Person endpoints ===\nSkipped (no referee/person id found; run the Matches cell first)")




=== Person endpoints ===
Skipped (no referee/person id found; run the Matches cell first)
